In [1]:
import pandas as pd
import plotly.express as px
import plotly.subplots as sp


from pandas.core.groupby.generic import DataFrameGroupBy, DataFrame

# Presentacion PCD

Obtención del DF completo. Primero generamos un dataframe con todas las marcas unidas.


In [2]:
dfs = []
for brand in ["Asus", "Gigabyte", "MSI", "XFX", "Zotac"]:
    df = pd.read_csv(f"../data/{brand}.csv")
    df["brand"] = brand
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

## Gráfico de correlaciones

Se filtra en correlaciones de las gráficas de NVIDIA series 4000, y de AMD series 7000 (las últimas generaciones de tarjetas gráficas).


In [ ]:
nvidia = df[df["chipset"].str.contains("RTX 40", case=False, na=False)]
amd = df[df["chipset"].str.contains("RX 7", case=False, na=False)]

column = ["price", "memory", "core_clock", "core_boost_clock", "length", "user_score", "user_ratings_count"]
nvidia = nvidia[column].corr()
amd = amd[column].corr()

nvidia_plot = px.imshow(
    nvidia,
    text_auto=True,
    title="Correlation Heatmap of GPU Attributes",
    labels={"color": "Correlation"},
    x=column,
    y=column,
    color_continuous_scale="Viridis",
    width=700,
    height=700
)

amd_plot = px.imshow(
    amd,
    text_auto=True,
    title="Correlation Heatmap of GPU Attributes",
    labels={"color": "Correlation"},
    x=column,
    y=column,
    color_continuous_scale="Viridis",
    width=700,
    height=700
)

# Create a subplot
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=["NVIDIA", "AMD"])
fig.add_trace(nvidia_plot.data[0], row=1, col=1)
fig.add_trace(amd_plot.data[0], row=1, col=2)

fig.update_layout(title="Correlation Heatmap of GPU Attributes", width=1500, height=700)
fig.show()

## Gráfico de Rendimiento vs Precio de las marcas por chipset


In [7]:
def standardize(column):
    return (column - column.mean()) / column.std()

In [ ]:
chipset = "GeForce RTX 4070 SUPER"

# Filter the data for the given chipset
chipset_filtered = df[df["chipset"] == chipset][["price", "core_boost_clock", "brand"]].reset_index(drop=True)
chipset_filtered

Podemos ver que todos los valores están presentes, excepto por algunos precios. Eliminaré las filas que no tengan precio.


In [ ]:
chipset_filtered = chipset_filtered[["price", "core_boost_clock", "brand"]].dropna()
chipset_filtered

In [ ]:
chipset_filtered.loc[:, "price"] = standardize(chipset_filtered["price"])
chipset_filtered.loc[:, "core_boost_clock"] = standardize(chipset_filtered["core_boost_clock"])
chipset_filtered.head()

Finalmente, podemos calcular la puntuación y graficar.


In [ ]:
chipset_filtered["punt"] = chipset_filtered["core_boost_clock"] / chipset_filtered["price"]

# Calculamos la media de la puntuación por marca
grouped = chipset_filtered.groupby("brand").mean().sort_values("punt", ascending=False)

fig = px.bar(
    grouped,
    x=grouped.index,
    y="punt",
    title=f"Puntuation of {chipset}",
    labels={"punt": "Puntuation", "index": "Brand"},
    width=800,
    height=600
)
fig.show()

# ¿Qué es mejor? ¿NVIDIA o AMD?

Haremos un gráfico de líneas donde comparamos el rendimiento de las últimas dos generaciones de las tarjetas gráficas de NVIDIA y AMD. Así podremos ver cuál es mejor en términos de rendimiento, además de poder tener un punto de referencia para comparar la calidad de las tarjetas gráficas de ambas marcas.


In [3]:
nvidia4000 = df[df["chipset"].str.contains("RTX 40", case=False, na=False)]
nvidia3000 = df[df["chipset"].str.contains("RTX 30", case=False, na=False)]
amd7000 = df[df["chipset"].str.contains("RX 7", case=False, na=False)]
amd6000 = df[df["chipset"].str.contains("RX 6", case=False, na=False)]

nvidia = pd.concat([nvidia4000, nvidia3000], ignore_index=True).reset_index(drop=True)
amd = pd.concat([amd7000, amd6000], ignore_index=True).reset_index(drop=True)

In [ ]:
nvidia.head()

In [ ]:
amd.head()

Ahora, nos quedamos con las columnas que vamos a usar, chipset, price, memory, core_clock y core_boost_clock. Y nos quitamos los que tengan valores nulos.


In [4]:
cols = ["chipset", "price", "core_clock", "core_boost_clock", "memory"]
nvidia = nvidia[cols].dropna()
amd = amd[cols].dropna()

In [5]:
def process_group(group: DataFrameGroupBy) -> DataFrame:
    group.loc[:, "price"] = standardize(group["price"])
    group.loc[:, "core_boost_clock"] = standardize(group["core_clock"])
    group["punt"] = group["core_boost_clock"] / group["price"]

    return group

Agrupamos por chipset, estandarizamos valores, calculamos el mismo puntaje de antes, y nos quedamos con el mejor chipeset de cada marca.


In [8]:
nvidia_processed = nvidia.groupby("chipset").apply(process_group)
amd_processed = amd.groupby("chipset").apply(process_group)

# max_nvidia = nvidia_processed.groupby("chipset")["punt"].max()

/tmp/ipykernel_13970/1047440425.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nvidia_processed = nvidia.groupby("chipset").apply(process_group)
/tmp/ipykernel_13970/1047440425.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  amd_processed = amd.groupby("chipset").apply(process_group)
